# Billboard Top 100

This dataset, available via Data.world, contains every weekly Hot 100 singles chart between 8/2/1958 and 12/28/2019 from Billboard.com. Each row of data represents a song and the corresponding position on that week's chart. Included in each row are the following elements:
1. Billboard Chart URL
2. WeekID
3. Song name
4. Performer name
5. SongID - Concatenation of song & performer
6. Current week on chart
7. Instance (this is used to separate breaks on the chart for a given song. Example, an instance of 6 tells you that this is the sixth time this song has appeared on the chart)
8. Previous week position
9. Peak Position (as of the corresponding week)
10. Weeks on Chart (as of the corresponding week)

Also available is a dataset containing each song's "audio features."

These include common attributes like genre as well as the following:
1. Danceability
2. Energy
3. Key
4. Loudness
5. Mode
6. Speechiness
7. Acousticness
8. Instrumentalness
9. Liveness
10. Valence
11. Tempo
12. Time Signature

Definitions of those terms are available at https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

Here's an example:

Liveliness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.



I'm excited about this data set. In addition to already numeric things like tempo, the ability to measure things like instrumentalness and speechiness give us some fun quantative measurements to play around with. I'm interested in looking for trends over time as well as what clusters of music might exist. It might be interesting to see if there's some seasonality to songs that spend the most time at the top, that is, is a song more likely to stay on the chart if it's released in the winter? I think there's a lot to explore here. Though admittedly, with so much to explore, I might have to be really careful with how I tidy the data. Choices I make in handling the data will make some questions easier to answer and some harder.

In [1]:
import numpy as np
import pandas as pd

In [22]:
df1= pd.read_csv("Hot Stuff.csv")
print(len(df1))
print(df1.dtypes)
df1.head()

320495
url                        object
WeekID                     object
Week Position               int64
Song                       object
Performer                  object
SongID                     object
Instance                    int64
Previous Week Position    float64
Peak Position               int64
Weeks on Chart              int64
dtype: object


,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1958-0...,8/2/1958,1,Poor Little Fool,Ricky Nelson,Poor Little FoolRicky Nelson,1,NaN,1,1
1,http://www.billboard.com/charts/hot-100/1995-1...,12/2/1995,1,One Sweet Day,Mariah Carey & Boyz II Men,One Sweet DayMariah Carey & Boyz II Men,1,NaN,1,1
2,http://www.billboard.com/charts/hot-100/1997-1...,10/11/1997,1,Candle In The Wind 1997/Something About The Wa...,Elton John,Candle In The Wind 1997/Something About The Wa...,1,NaN,1,1
3,http://www.billboard.com/charts/hot-100/2006-0...,7/1/2006,1,Do I Make You Proud,Taylor Hicks,Do I Make You ProudTaylor Hicks,1,NaN,1,1
4,http://www.billboard.com/charts/hot-100/2009-1...,10/24/2009,1,3,Britney Spears,3Britney Spears,1,NaN,1,1


In [25]:
df1['WeekID']=pd.to_datetime(df1['WeekID'], infer_datetime_format=True) #Let's convert WeekID to a date, then we can use max functions to find the most recent WeekID

In [29]:
df1.isnull().sum()

url                           0
WeekID                        0
Week Position                 0
Song                          0
Performer                     0
SongID                        0
Instance                      0
Previous Week Position    30784
Peak Position                 0
Weeks on Chart                0
dtype: int64

This is actually a really clean set. Those missing Previous Week Position values are all from songs appearing on the chart for the first time and shouldn't have a value there. I'm going to tidy this data by defining an observation to be the number of weeks a song was on the chart, it's peak position, and it's final Week ID

In [30]:
df1['Song']

0                                          Poor Little Fool
1                                             One Sweet Day
2         Candle In The Wind 1997/Something About The Wa...
3                                       Do I Make You Proud
4                                                         3
                                ...                        
320490                                     Jingle Bell Rock
320491                                     Jingle Bell Rock
320492                                     Jingle Bell Rock
320493                                     Jingle Bell Rock
320494                                     Jingle Bell Rock
Name: Song, Length: 320495, dtype: object

Hmm, a few occurences of Jingle Bell Rock. Let's see how many and when they appear.

In [40]:
df1[(df1['Song']=='Jingle Bell Rock')].sort_values(by='Weeks on Chart', ascending = False)

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
320494,https://www.billboard.com/charts/hot-100/2019-...,2019-12-28,9,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,15.0,9,29
320493,https://www.billboard.com/charts/hot-100/2019-...,2019-12-21,15,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,23.0,15,28
320492,https://www.billboard.com/charts/hot-100/2019-...,2019-12-14,23,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,47.0,23,27
320491,https://www.billboard.com/charts/hot-100/2019-...,2019-12-07,47,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,NaN,47,26
320490,https://www.billboard.com/charts/hot-100/2019-...,2019-01-05,8,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,13.0,8,25
320489,https://www.billboard.com/charts/hot-100/2018-...,2018-12-29,13,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,15.0,13,24
320488,https://www.billboard.com/charts/hot-100/2018-...,2018-12-22,15,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,26.0,15,23
320487,https://www.billboard.com/charts/hot-100/2018-...,2018-12-15,26,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,33.0,26,22
320486,https://www.billboard.com/charts/hot-100/2018-...,2018-12-08,33,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,NaN,33,21
320470,http://www.billboard.com/charts/hot-100/2017-0...,2017-01-07,29,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,6,46.0,29,20


So in addition to there being two different versions of Jingle Bell Rock, it looks like 'Weeks on Chart' allows for nonconsecutive weeks. Bobby Helms jumped from 1962 to 2016 and then showed up as recently as 2018. In a separate analysis, I might be interested in how Christmas music has changed over time and look for evidence of how long it takes pop music trends to work their way into the most played Christmas songs. I think Christmas music is so specific though, that I'm going to disregard or remove it from the dataframe at some point. I notice the second file has genre information. I'll look to see if I can use that. I could always scrap December, or songs that originate in late November/early December. To that end, I could also use the point where Christmas music starts up to measure "Christmas Creep," the feeling that the Christmas season begins earlier each year.

In [41]:
df1[(df1['Song']=='Jingle Bell Rock') & (df1['Performer'] =='Bobby Helms')].sort_values(by='Weeks on Chart', ascending = False)

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
320494,https://www.billboard.com/charts/hot-100/2019-...,2019-12-28,9,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,15.0,9,29
320493,https://www.billboard.com/charts/hot-100/2019-...,2019-12-21,15,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,23.0,15,28
320492,https://www.billboard.com/charts/hot-100/2019-...,2019-12-14,23,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,47.0,23,27
320491,https://www.billboard.com/charts/hot-100/2019-...,2019-12-07,47,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,8,NaN,47,26
320490,https://www.billboard.com/charts/hot-100/2019-...,2019-01-05,8,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,13.0,8,25
320489,https://www.billboard.com/charts/hot-100/2018-...,2018-12-29,13,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,15.0,13,24
320488,https://www.billboard.com/charts/hot-100/2018-...,2018-12-22,15,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,26.0,15,23
320487,https://www.billboard.com/charts/hot-100/2018-...,2018-12-15,26,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,33.0,26,22
320486,https://www.billboard.com/charts/hot-100/2018-...,2018-12-08,33,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,7,NaN,33,21
320470,http://www.billboard.com/charts/hot-100/2017-0...,2017-01-07,29,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,6,46.0,29,20


In [56]:
new_df1 = df1.groupby('SongID').agg({'WeekID':'max', 'Peak Position':'min', 'Weeks on Chart':'max' })[['WeekID', 'Peak Position','Weeks on Chart']].reset_index()

In [62]:
new_df1.head()

,SongID,WeekID,Peak Position,Weeks on Chart
0,"""B"" GirlsYoung And Restless",1990-08-11,54,15
1,"""Cherry Cherry"" from Hot August NightNeil Diamond",1973-05-19,31,10
2,"""Having A Party"" MedleyThe Ovations (Featuring...",1973-12-08,56,9
3,"""Joy"" Pt. IIsaac Hayes",1974-02-16,30,9
4,"""Roots"" MedleyQuincy Jones",1977-04-16,57,7


In [61]:
new_df1.isnull().sum()

SongID            0
WeekID            0
Peak Position     0
Weeks on Chart    0
dtype: int64

In [58]:
df2 = pd.read_excel("Song_data.xlsx")
df2.head()

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.37100,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,...,6.0,-4.169,1.0,0.2100,0.05880,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,...,8.0,-5.725,0.0,0.1680,0.00124,0.000001,0.0716,0.856,135.979,4.0


In [36]:
df2.sort_values('SongID')

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
10308,"""B"" GirlsYoung And Restless",Young And Restless,"""B"" Girls",[],0rWUHulFlScxQsCq7kWKde,https://p.scdn.co/mp3-preview/c551cbcaadd17bf4...,Something To Get You Hyped,0.0,242106.0,20.0,...,7.0,-11.913,1.0,0.4390,0.0160,0.000000,0.3120,0.769,193.762,3.0
12928,"""Cherry Cherry"" from Hot August NightNeil Diamond",Neil Diamond,"""Cherry Cherry"" from Hot August Night","['adult standards', 'brill building pop', 'fol...",18ocy9Be9SxWEEnOLGDfkS,NaN,Hot August Night / NYC (Live From Madison Squa...,0.0,443320.0,16.0,...,9.0,-5.061,1.0,0.1370,0.0941,0.000907,0.8670,0.604,172.349,4.0
19355,"""Having A Party"" MedleyThe Ovations (Featuring...",The Ovations (Featuring Louis Williams),"""Having A Party"" Medley","['memphis soul', 'southern soul']",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11563,"""Joy"" Pt. IIsaac Hayes",Isaac Hayes,"""Joy"" Pt. I","['classic soul', 'disco', 'funk', 'jazz funk',...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19935,"""Roots"" MedleyQuincy Jones",Quincy Jones,"""Roots"" Medley","['adult standards', 'disco', 'funk', 'jazz fun...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,teachmeMusiq Soulchild,Musiq Soulchild,teachme,"['alternative r&b', 'dance pop', 'neo soul', '...",0ODFPypnJLE6hKgahiuSzn,https://p.scdn.co/mp3-preview/0a49d387a7ecc54d...,Luvanmusiq,0.0,260017.0,57.0,...,8.0,-5.110,0.0,0.1350,0.4730,0.000000,0.0722,0.436,172.969,3.0
25353,whoa (mind in awe)XXXTENTACION,XXXTENTACION,whoa (mind in awe),"['emo rap', 'miami hip hop']",7pdF27mSDuPWhppnHAmWHa,https://p.scdn.co/mp3-preview/f69b47e8941ef8f4...,SKINS,0.0,157776.0,77.0,...,6.0,-2.939,0.0,0.0441,0.6530,0.918000,0.1010,0.369,160.147,4.0
25358,whoknowsMusiq,Musiq,whoknows,"['alternative r&b', 'dance pop', 'neo soul', '...",69rOfKSz0ys8UdgTkGd6qV,NaN,soulstar,0.0,295320.0,39.0,...,2.0,-6.144,1.0,0.1220,0.2300,0.000000,0.3390,0.316,118.862,4.0
25845,www.memoryAlan Jackson,Alan Jackson,www.memory,"['contemporary country', 'country', 'country r...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df2.columns

Index(['SongID', 'Performer', 'Song', 'spotify_genre', 'spotify_track_id',
       'spotify_track_preview_url', 'spotify_track_album',
       'spotify_track_explicit', 'spotify_track_duration_ms',
       'spotify_track_popularity', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'],
      dtype='object')

I can use 'SongID' to join the dataframes. I'd like to clean the genre information up a bit. I don't need to keep the spotify information, but I want everything from danceability to time_signature.

### Work on next time:

df2:
    expand genre column
    remove columns
    join data
    deal with missing information
    look for trends in numeric values
    